In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
!pip install -q -r ../requirements.dev.txt

## Imports & Functions

In [69]:
# system lib
import os
import sys
from datetime import datetime

# append package path
LIB_PATHS = [
    os.path.join(os.getcwd(), "..")
]

for _lib in LIB_PATHS:
    if _lib not in sys.path:
        sys.path.append(_lib)
        
# import python packages  
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
  
# import local packages
from src.constant import Chain
from src.process_queue import TABLE_NAME_MAPPER
from src.utils import print_log

# load dotenv
if load_dotenv():
    print_log(".env file loaded")
    
CHAINS = [
    Chain.ARBITRUM_ONE, Chain.OPTIMISM, 
    Chain.BNB_CHAIN, Chain.POLYGON, Chain.GNOSIS]

.env file loaded


In [63]:
def save_cache(chain: Chain, cache_path: str = "cache", reload: bool = True) -> None:
    os.makedirs(cache_path, exist_ok=True)
    cache_path = os.path.join(cache_path, Chain.resolve_connext_domain(chain)) + ".csv"
    if os.path.exists(cache_path) and not reload:
        print_log("Cache already existed!")
        return
    else:
        print_log("Reloading data...")
        
    table_name = TABLE_NAME_MAPPER.get(chain)
    cnx = create_engine(
        f"mysql+pymysql://{os.getenv('AWS_RDS_USERNAME')}:{os.getenv('AWS_RDS_PASSWORD')}@{os.getenv('AWS_RDS_HOSTNAME')}/connext")
    df = pd.read_sql(
        f"SELECT * FROM {table_name}",
        cnx
    )
    df.to_csv(cache_path, index=False)
    print_log("Cache saved successfully!")

    
def load_txn_data(chain: Chain, use_cache: bool = True, cache_path: str = "cache") -> pd.DataFrame:
    cache_path = os.path.join(cache_path, Chain.resolve_connext_domain(chain)) + ".csv"
    if use_cache and os.path.exists(cache_path):
        print_log("Cache loaded successfully!")
        df = pd.read_csv(cache_path)
    else:
        print_log("Cache not found, loading from AWS RDS")
        table_name = TABLE_NAME_MAPPER.get(chain)
        cnx = create_engine(
            f"mysql+pymysql://{os.getenv('AWS_RDS_USERNAME')}:{os.getenv('AWS_RDS_PASSWORD')}@{os.getenv('AWS_RDS_HOSTNAME')}/connext")
        df = pd.read_sql(
            f"SELECT * FROM {table_name}",
            cnx
        )
    df = df.drop("id", axis=1).drop_duplicates()
    return df

## Load Data

In [65]:
for chain in CHAINS:
    save_cache(chain, reload=False)

dataset = {}
for chain in CHAINS:
    df = load_txn_data(chain)
    print_log(f"Data loaded from chain {Chain.resolve_connext_domain(chain)}, total of {len(df)} record")
    dataset[chain] = df

Cache already existed!
Cache already existed!
Cache already existed!
Cache already existed!
Cache already existed!
Cache loaded successfully!
Data loaded from chain arbitrum_one, total of 23471 record
Cache loaded successfully!
Data loaded from chain optimism, total of 6163 record
Cache loaded successfully!
Data loaded from chain bnb_chain, total of 3399 record
Cache loaded successfully!
Data loaded from chain polygon, total of 13344 record
Cache loaded successfully!
Data loaded from chain gnosis, total of 2056 record


## Analysis

In [66]:
chain = Chain.POLYGON

In [67]:
df = dataset[chain]

In [68]:
df

,timestamp,chain,hash,user_address,token_address,token_amount,action
0,1672780925,1886350457,0xfedf86d20dca8062f2c0619e74557fb09df875416122...,0xade09131c6f43fe22c2cbabb759636c43cfc181e,0xa03258b76Ef13AF716370529358f6A79eb03ec12,1.00000,2
1,1672815083,1886350457,0x31185a9751f885655e2324ed59551510823a5e9f207e...,0xd78c323477500744214f662c15c82895a0bbdae9,0xa03258b76Ef13AF716370529358f6A79eb03ec12,49993.40000,1
2,1672930555,1886350457,0x6c56fae03cfc751f3d4539b9fc03bb0fd7cc7327aa42...,0x09c6fedfbbc7861b95c3be56df9bff2328392970,0xa03258b76Ef13AF716370529358f6A79eb03ec12,1.99795,1
3,1672942790,1886350457,0x99694fe9a2098905298f1f3ca97a4a8300cd3d0fface...,0x560034a84358e18ffaf5f25ee3f32107cf4f7eed,0xa03258b76Ef13AF716370529358f6A79eb03ec12,5.00059,1
4,1672958258,1886350457,0x97cc5e16ead3ff46a4a91fd136c5d036e223da84094e...,0xfe264f8a5f66a750f9a24b69f3e27b88ea8632c4,0xa03258b76Ef13AF716370529358f6A79eb03ec12,1.98784,1
...,...,...,...,...,...,...,...
13518,1681873840,1886350457,0xe8ca7de951a00ab9e0b8bd86f984840794804bdedfe7...,0xfec8ac88aa938d32b54ca92a620d81f1a1b45b15,0xa03258b76Ef13AF716370529358f6A79eb03ec12,95.62950,1
13519,1681874087,1886350457,0xe251651a317a3d868c46553a8816e700639ddf42ef82...,0xf0116097dadadf896318554ebd22237a214ff6ca,0xa03258b76Ef13AF716370529358f6A79eb03ec12,398.15100,1
13520,1681874284,1886350457,0x2c85dd83c0abbe3dfba00ca1ac51968199c34330fb3d...,0x06c4d97162283327df694b4cbabb4c2ad823c24c,0xa03258b76Ef13AF716370529358f6A79eb03ec12,88.36540,1
13521,1681874858,1886350457,0x2614167ee9b1611254fcddf0d485805511f846d7fa09...,0xa33f0a402d9cff2e4e6f03698161756f60723f56,0xa03258b76Ef13AF716370529358f6A79eb03ec12,95.30610,1
